<a href="https://colab.research.google.com/github/sezerblt/dataset_repository/blob/main/dataset_02_skincancer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Verisetinin bulunduğu klasöre gitmek için os modulu import edilip aut yetkisinden sonra işlemi başlatıyoruz

görüntü veri setinin bulundugu klasör git ve gerekli os ayarlamaları yap

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
import os
os.environ["KAGGLE_CONFIG_DIR"]="/content/gdrive/MyDrive/Kaggle"

Kullanacağımız tüm paketleri import ediyoruz.İşlemleri yapacağımız dizine gidiyoruz

In [ ]:
import numpy as np
import os
import tensorflow as tf
import tensorflow_datasets as tfds
import pathlib
from PIL import Image
%cd "/content/gdrive/MyDrive/Kaggle"

VERİ SETİNİ LİNK ÜZERİNDEN İNDİRME

#link üzeinden dosya indirme


Görüntünün bulunduğu dizin ve ve dosya içerisinde bulunan görüntü sayısını bir değere atayalım
 Test ve train verilerini ayıralım

In [ ]:
file_path="/content/gdrive/MyDrive/Kaggle/data"
data_dir = pathlib.Path(file_path)
imageCount=len(list(data_dir.glob("*/*/*.jpg")))

TRAIN_bening    = file_path+str("/train/benign")
TRAIN_malignant = file_path+str("/train/malignant")
TEST_bening     = file_path+str("/test/benign")
TEST_malignant  = file_path+str("/test/malignant")

TRAIN_bening_list   = list(data_dir.glob("train/benign/*.jpg"))
TRAIN_malignant_list= list(data_dir.glob("train/malignant/*.jpg"))
TEST_bening_list    = list(data_dir.glob("test/benign/*.jpg"))
TEST_malignant_list = list(data_dir.glob("test/malignant/*.jpg"))


görüntü dataseti yükleyip, matrise dönüstürelim

In [ ]:
joinLambda =lambda image_name : np.asarray(Image.open(image_name).convert("RGB"))
print("...")
IMG_train_benign = [joinLambda(os.path.join(TRAIN_bening, filename)) for filename in os.listdir(TRAIN_bening)]
X_BENIGN = np.array(IMG_train_benign, dtype='uint8')

IMG_train_malignant = [joinLambda(os.path.join(TRAIN_malignant, filename)) for filename in os.listdir(TRAIN_malignant)]
X_MALIGNANT = np.array(IMG_train_malignant, dtype='uint8')
print("...")

In [ ]:
IMG_test_benign = [joinLambda(os.path.join(TEST_bening, filename)) for filename in os.listdir(TEST_bening)]
X_BENIGN_TEST = np.array(IMG_test_benign, dtype='uint8')

IMG_test_malignant = [joinLambda(os.path.join(TEST_malignant, filename)) for filename in os.listdir(TEST_malignant)]
X_MALIGNANT_TEST = np.array(IMG_test_malignant, dtype='uint8')
print("...")

Sınıflandırma için etiketleri tanımlayalım.
zeros:Kotu huylu degil
ones:Kotu Huylu

In [ ]:
Y_Benign    = np.zeros(X_BENIGN.shape[0])
Y_Malignant = np.ones(X_MALIGNANT.shape[0])

Y_TEST_Benign    = np.zeros(X_BENIGN_TEST.shape[0])
Y_TEST_Malignant = np.ones(X_MALIGNANT_TEST.shape[0])

Egitim için iyi huylu ve kotu huylu verilerini birleştir

Egitim için iyi huylu ve kotu huylu etiketlerini  birleştir

TEST için iyi huylu ve kotu huylu verilerini birleştir

TEST için iyi huylu ve kotu huylu etiketlerini  birleştir


In [ ]:
X_TRAIN = np.concatenate((X_BENIGN,X_MALIGNANT),axis=0)
Y_TRAIN = np.concatenate((Y_Benign,Y_Malignant),axis=0)


X_TEST = np.concatenate((X_BENIGN_TEST,X_MALIGNANT_TEST),axis=0)
Y_TEST = np.concatenate((Y_TEST_Benign,Y_TEST_Malignant),axis=0)

Verileri karıştır.shuffling)

In [ ]:
len_train = np.arange(X_TRAIN.shape[0])
np.random.shuffle(len_train)
X_TRAIN=X_TRAIN[len_train]
Y_TRAIN=Y_TRAIN[len_train]

len_train = np.arange(X_TEST.shape[0])
np.random.shuffle(len_train)
X_TEST =X_TEST[len_train]
Y_TEST =Y_TEST[len_train]

veri seti boyutlarını ekranda goster

In [ ]:
print("train image shape:",X_TRAIN.shape)
print("test image shape:",X_TEST.shape)
print("len train image :",len(X_TRAIN))
print("")
print("train label shape:",Y_TRAIN.shape)
print("test label shape:",Y_TEST.shape)
print("len train label :",len(Y_TRAIN))

In [ ]:
class_names=["Iyi","kotu"]

örnek 25 tane esmi görselleştirelim

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.xticks([])
  plt.yticks([])
  plt.grid(False)
  plt.imshow(X_TRAIN[i], cmap=plt.cm.binary)
  if Y_TRAIN[i]==1:
    plt.xlabel(class_names[int(Y_TRAIN[i])]+" huylu")
  else:
    plt.xlabel(class_names[int(Y_TRAIN[i])]+" huylu")
plt.show()


In [ ]:
plt.bar(0, Y_TRAIN[np.where(Y_TRAIN == 0)].shape[0], label = 'iyi huylu')
plt.bar(1, Y_TRAIN[np.where(Y_TRAIN == 1)].shape[0], label = 'kotu huylu')
plt.legend()
plt.title("Training Data")
plt.show()

plt.bar(0, Y_TEST[np.where(Y_TEST == 0)].shape[0], label = 'benign')
plt.bar(1, Y_TEST[np.where(Y_TEST == 1)].shape[0], label = 'malignant')
plt.legend()
plt.title("Test Data")
plt.show()

In [ ]:

H,W,C=X_TRAIN.shape[1],X_TRAIN.shape[2],X_TRAIN.shape[3]
print("h: ",H)
print("w: ",W)
print("c: ",C)

model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(H, W,C)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_TRAIN, Y_TRAIN, epochs=5)

In [ ]:
test_loss, test_acc = model.evaluate(X_TEST,  Y_TEST, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
probability_model = tf.keras.Sequential([model, 
                                         tf.keras.layers.Softmax()])
predictions = probability_model.predict(X_TEST)

for i in range(5):
  a=np.random.randint(20)
  print(predictions[a])
  print("\n")

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  true_label, img = true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("Gercek:{} {:2.0f}% ".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  true_label = true_label[i]
  plt.grid(False)
  plt.xticks(range(2))
  plt.yticks([])
  thisplot = plt.bar(range(2), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  

  thisplot[predicted_label].set_color('red')
  thisplot[int(true_label)].set_color('blue')
  plt.xlabel("Tahmin:({})".format(
                                class_names[int(true_label)],
                                color="c")
  )

num_rows = 5
num_cols = 2
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], Y_TEST, X_TEST)

  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], Y_TEST)
plt.tight_layout()
plt.show()